In [301]:
# Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten



# Additional Libraries
import scipy
import statsmodels.api as sm
import xgboost as xgb

import cv2

# Utility Libraries
import joblib
import pickle


In [302]:
df=pd.read_csv('train.csv', na_values=['\\N'])

C:\Users\risha\AppData\Local\Temp\ipykernel_13404\1670193474.py:1: DtypeWarning: Columns (15,23,37,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('train.csv', na_values=['\\N'])


In [303]:
df.shape[0]

2830101

In [304]:
df.sample(5)

,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
2199981,1398,84,21,4,6.0,2,4.0,4,4,5.0,...,8.0,11,11,0,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,Finished,15183678
2514623,20348,338,9,4,11.0,9,2.0,2,2,18.0,...,0.0,23,23,0,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,Finished,267006456
225694,16440,670,376,195,15.0,12,6.0,6,6,1.0,...,1.0,20,20,0,cooper-brm,Cooper-BRM,British,http://en.wikipedia.org/wiki/Cooper_Car_Company,+2 Laps,820849200
867355,10997,464,163,6,28.0,3,2.0,2,2,6.0,...,11.0,9,9,0,ferrari,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Finished,255658256
2240669,846,57,11,8,22.0,20,NaN,R,16,0.0,...,0.0,21,21,0,super_aguri,Super Aguri,Japanese,http://en.wikipedia.org/wiki/Super_Aguri_F1,Hydraulics,10124928


In [305]:
df.isnull().sum()

resultId                        0
racerId                         0
driverId                        0
constructorId                   0
number                        160
grid                            0
position_x                1125357
positionText_x                  0
positionOrder                   0
points                          0
laps                            0
time_x                    1871686
timetaken_in_millisec     1871741
fastestLap                2112010
rank                      2105375
fastestLapTime            2112010
max_speed                 2112010
statusId                        0
year                            0
round                           0
circuitId                       0
grand_prix                      0
date                            0
time_y                    2150026
url_x                           0
fp1_date                  2830101
fp1_time                  2830101
fp2_date                  2830101
fp2_time                  2830101
fp3_date      

In [306]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 55 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   resultId                int64  
 1   racerId                 int64  
 2   driverId                int64  
 3   constructorId           int64  
 4   number                  float64
 5   grid                    int64  
 6   position_x              float64
 7   positionText_x          object 
 8   positionOrder           int64  
 9   points                  float64
 10  laps                    int64  
 11  time_x                  object 
 12  timetaken_in_millisec   float64
 13  fastestLap              float64
 14  rank                    float64
 15  fastestLapTime          object 
 16  max_speed               float64
 17  statusId                int64  
 18  year                    int64  
 19  round                   int64  
 20  circuitId               int64  
 21  grand_prix              object 

In [307]:
df['positionText_x'].sample(10)

536136     4
2632929    9
2292175    5
1975395    R
1698631    R
2482031    R
68491      R
2416141    9
625412     R
1096517    5
Name: positionText_x, dtype: object

In [308]:
x = df['time_x'].mode()[0]
x

'+17.456'

In [309]:
df['time_x'] = df['time_x'].fillna(x)
df['time_x'].isnull().sum()

0

In [310]:
x = df['timetaken_in_millisec'].mode()[0]
df['timetaken_in_millisec'] = df['timetaken_in_millisec'].fillna(x)
df['timetaken_in_millisec'].isnull().sum()

0

In [311]:
x = df['fastestLap'].mode()[0]
df['fastestLap'] = df['fastestLap'].fillna(x)
df['fastestLap'].isnull().sum()

0

In [312]:
x = df['rank'].mode()[0]
df['rank'] = df['rank'].fillna(x)
df['rank'].isnull().sum()

0

In [313]:
x = df['fastestLapTime'].mode()[0]
df['fastestLapTime'] = df['fastestLapTime'].fillna(x)
df['fastestLapTime'].isnull().sum()

0

In [314]:
x = df['max_speed'].mean()
df['max_speed'] = df['max_speed'].fillna(x)
df['max_speed'].isnull().sum()

0

In [315]:
df1=df.drop(columns=['url_x','url_y','url','driver_num','driver_code','position_x','positionText_x','result_driver_standing','time_y','fp1_date','fp1_time','fp2_date','fp2_time','fp3_date','fp3_time','quali_date','quali_time','sprint_date','sprint_time','number','time_x'],axis=1)

In [316]:
df1['full_name'] = df1['forename'] + ' ' + df1['surname']

# Drop 'forename' and 'surname' columns if needed
df1 = df1.drop(columns=['forename', 'surname'])

In [317]:
df1.sample(5)

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLap,...,raceId_y,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,status,full_name
890214,10308,439,163,6,6,2,6.0,60,5827095.0,52.0,...,451,27.0,6,6,0,ferrari,Ferrari,Italian,Finished,René Arnoux
1825401,2941,158,18,3,21,11,0.0,46,4584572.0,52.0,...,352,189.0,5,5,2,williams,Williams,British,Engine,Jenson Button
1821109,3638,190,23,3,9,5,2.0,53,5578279.0,52.0,...,158,4.0,3,3,0,williams,Williams,British,Finished,Ralf Schumacher
1761841,3940,204,56,6,5,2,6.0,53,4668277.0,52.0,...,128,3.0,9,9,0,ferrari,Ferrari,Italian,Finished,Eddie Irvine
2199711,1397,84,31,1,4,3,6.0,58,5094089.0,39.0,...,68,26.0,8,8,0,mclaren,McLaren,British,Finished,Juan Pablo Montoya


In [318]:
df1['full_name'].value_counts()

full_name
Rubens Barrichello    100408
Michael Schumacher     96404
Fernando Alonso        80352
Jenson Button          79608
Kimi Räikkönen         68482
                       ...  
George Constantine         1
Robert La Caze             1
Peter Ryan                 1
Ken Miles                  1
Bob Drake                  1
Name: count, Length: 816, dtype: int64

In [319]:
from datetime import datetime
# Convert 'dob' to datetime format
df1['dob'] = pd.to_datetime(df1['dob'])

# Create a reference date for each race
df1['race_date'] = pd.to_datetime(df1['year'], format='%Y')

# Calculate age at the time of race
df1['age_at_race'] = df1['race_date'].dt.year - df1['dob'].dt.year

In [320]:
df1.tail()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLap,...,position,positionText_y,wins,constructorRef,company,nationality_y,status,full_name,race_date,age_at_race
2830096,22109,899,17,9,4,2,18.0,71,5566752.0,51.0,...,4,4,2,red_bull,Red Bull,Austrian,Finished,Mark Webber,2013-01-01,37
2830097,22109,899,17,9,4,2,18.0,71,5566752.0,51.0,...,4,4,2,red_bull,Red Bull,Austrian,Finished,Mark Webber,2013-01-01,37
2830098,22109,899,17,9,4,2,18.0,71,5566752.0,51.0,...,8,8,0,red_bull,Red Bull,Austrian,Finished,Mark Webber,2013-01-01,37
2830099,22109,899,17,9,4,2,18.0,71,5566752.0,51.0,...,10,10,0,red_bull,Red Bull,Austrian,Finished,Mark Webber,2013-01-01,37
2830100,22109,899,17,9,4,2,18.0,71,5566752.0,51.0,...,8,8,0,red_bull,Red Bull,Austrian,Finished,Mark Webber,2013-01-01,37


In [321]:
import category_encoders as ce

In [322]:
df1 = df1.drop(columns=['dob', 'race_date'])
# List of high-cardinality columns to encode
high_cardinality_columns = ['full_name', 'constructorRef', 'nationality_y','company','grand_prix','nationality','status']

# Initialize the target encoder
target_encoder = ce.TargetEncoder(cols=high_cardinality_columns)

# Apply target encoding to the specified columns
df1_encoded = target_encoder.fit_transform(df1[high_cardinality_columns], df1['position'])

# Replace original columns with encoded columns in df1
df1[high_cardinality_columns] = df1_encoded

 

In [323]:
# Columns to drop
columns_to_drop = ['resultId', 'racerId', 'driverId', 'constructorId','date', 'driverRef', 'driverStandingsId','positionText_y','year','raceId_y','statusId']

# Drop columns from df1 if they exist
df1 = df1.drop(columns=[col for col in columns_to_drop if col in df1.columns])

In [324]:
df1

,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLap,rank,fastestLapTime,max_speed,round,...,nationality,points_y,position,wins,constructorRef,company,nationality_y,status,full_name,age_at_race
0,1,1,9.0,70,8003600.0,52.0,1.0,1:14.117,202.943395,1,...,15.452703,3.33,3,0,15.873309,15.873309,12.330653,9.155723,5.270833,44
1,1,1,9.0,70,8003600.0,52.0,1.0,1:14.117,202.943395,1,...,15.452703,6.33,3,0,15.873309,15.873309,12.330653,9.155723,5.270833,44
2,1,1,9.0,70,8003600.0,52.0,1.0,1:14.117,202.943395,1,...,15.452703,6.33,4,0,15.873309,15.873309,12.330653,9.155723,5.270833,44
3,1,1,9.0,70,8003600.0,52.0,1.0,1:14.117,202.943395,1,...,15.452703,10.33,3,0,15.873309,15.873309,12.330653,9.155723,5.270833,44
4,1,1,9.0,70,8003600.0,52.0,1.0,1:14.117,202.943395,1,...,15.452703,10.33,4,0,15.873309,15.873309,12.330653,9.155723,5.270833,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830096,4,2,18.0,71,5566752.0,51.0,1.0,1:15.436,205.636000,19,...,10.799576,61.50,4,2,7.330218,7.330218,7.330218,9.155723,8.082517,37
2830097,4,2,18.0,71,5566752.0,51.0,1.0,1:15.436,205.636000,19,...,10.799576,69.50,4,2,7.330218,7.330218,7.330218,9.155723,8.082517,37
2830098,4,2,18.0,71,5566752.0,51.0,1.0,1:15.436,205.636000,19,...,10.799576,4.00,8,0,7.330218,7.330218,7.330218,9.155723,8.082517,37
2830099,4,2,18.0,71,5566752.0,51.0,1.0,1:15.436,205.636000,19,...,10.799576,6.00,10,0,7.330218,7.330218,7.330218,9.155723,8.082517,37


In [325]:
def convert_to_milliseconds(row):
    time_str = row['fastestLapTime']
    minutes, seconds = map(float, time_str.split(':'))
    return float((minutes * 60 + seconds)*1000)

df1['fastestLapTimeinMilliseconds'] = df1.apply(convert_to_milliseconds,axis=1)
df1['fastestLapTimeinMilliseconds']

0          74117.0
1          74117.0
2          74117.0
3          74117.0
4          74117.0
            ...   
2830096    75436.0
2830097    75436.0
2830098    75436.0
2830099    75436.0
2830100    75436.0
Name: fastestLapTimeinMilliseconds, Length: 2830101, dtype: float64

In [326]:
df1 = df1.drop(columns=['fastestLapTime'],axis=1)

In [327]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 22 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   grid                          int64  
 1   positionOrder                 int64  
 2   points                        float64
 3   laps                          int64  
 4   timetaken_in_millisec         float64
 5   fastestLap                    float64
 6   rank                          float64
 7   max_speed                     float64
 8   round                         int64  
 9   circuitId                     int64  
 10  grand_prix                    float64
 11  nationality                   float64
 12  points_y                      float64
 13  position                      int64  
 14  wins                          int64  
 15  constructorRef                float64
 16  company                       float64
 17  nationality_y                 float64
 18  status                

In [328]:
df1.columns

Index(['grid', 'positionOrder', 'points', 'laps', 'timetaken_in_millisec',
       'fastestLap', 'rank', 'max_speed', 'round', 'circuitId', 'grand_prix',
       'nationality', 'points_y', 'position', 'wins', 'constructorRef',
       'company', 'nationality_y', 'status', 'full_name', 'age_at_race',
       'fastestLapTimeinMilliseconds'],
      dtype='object')

In [329]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [330]:
x = df1[['fastestLapTimeinMilliseconds']]
scaler.fit(x)
df1['fastestLapTimeinMilliseconds'] = scaler.transform(x)
df1['fastestLapTimeinMilliseconds']

0          0.028188
1          0.028188
2          0.028188
3          0.028188
4          0.028188
             ...   
2830096    0.038188
2830097    0.038188
2830098    0.038188
2830099    0.038188
2830100    0.038188
Name: fastestLapTimeinMilliseconds, Length: 2830101, dtype: float64

In [331]:
x = df1[['timetaken_in_millisec']]
scaler.fit(x)
df1['timetaken_in_millisec'] = scaler.transform(x)
df1['timetaken_in_millisec']

0          0.479500
1          0.479500
2          0.479500
3          0.479500
4          0.479500
             ...   
2830096    0.300526
2830097    0.300526
2830098    0.300526
2830099    0.300526
2830100    0.300526
Name: timetaken_in_millisec, Length: 2830101, dtype: float64

In [332]:
df1.columns

Index(['grid', 'positionOrder', 'points', 'laps', 'timetaken_in_millisec',
       'fastestLap', 'rank', 'max_speed', 'round', 'circuitId', 'grand_prix',
       'nationality', 'points_y', 'position', 'wins', 'constructorRef',
       'company', 'nationality_y', 'status', 'full_name', 'age_at_race',
       'fastestLapTimeinMilliseconds'],
      dtype='object')

In [333]:
from sklearn.model_selection import train_test_split
import xgboost as xg
from sklearn.metrics import mean_squared_error, r2_score

# Assume X contains your features and y is your target variable 'age_at_race'
X = df1.drop(['position'], axis=1)
y = df1['position']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate Linear Regression model
model = xg.XGBRegressor(objective ='reg:linear', n_estimators = 200, seed = 123) 

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")


c:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:45:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


Mean Squared Error (MSE): 19.437533007120408
Root Mean Squared Error (RMSE): 4.408801765459682
R-squared (R2): 0.7922476530075073


c:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [334]:
y_pred

array([22.238485 ,  2.7668726,  3.5994895, ..., 15.178707 ,  9.793122 ,
        3.0494726], dtype=float32)

In [335]:
df_feat_importance = pd.DataFrame({'feat_name':X.columns.tolist(), 'model_feat_imp_train':model.feature_importances_}).sort_values(by='model_feat_imp_train', ascending=False)\
                                                                                                                        .reset_index(drop=True)
df_feat_importance

,feat_name,model_feat_imp_train
0,points_y,0.496960
1,wins,0.130564
2,full_name,0.115546
3,fastestLapTimeinMilliseconds,0.050145
4,rank,0.049510
5,grand_prix,0.025865
6,timetaken_in_millisec,0.022485
7,constructorRef,0.018185
8,nationality,0.017831
9,age_at_race,0.015349


In [336]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 21 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   grid                          int64  
 1   positionOrder                 int64  
 2   points                        float64
 3   laps                          int64  
 4   timetaken_in_millisec         float64
 5   fastestLap                    float64
 6   rank                          float64
 7   max_speed                     float64
 8   round                         int64  
 9   circuitId                     int64  
 10  grand_prix                    float64
 11  nationality                   float64
 12  points_y                      float64
 13  wins                          int64  
 14  constructorRef                float64
 15  company                       float64
 16  nationality_y                 float64
 17  status                        float64
 18  full_name             

In [337]:
# for test data 

In [338]:
df2=pd.read_csv('test.csv', na_values=['\\N'])

C:\Users\risha\AppData\Local\Temp\ipykernel_13404\444574744.py:1: DtypeWarning: Columns (25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv('test.csv', na_values=['\\N'])


In [339]:
df2

,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,driverStandingsId,raceId_y,points_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,23949,997,4,1,14,13,8.0,8,8,4.0,...,12474,93,16.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,298739826
1,23949,997,4,1,14,13,8.0,8,8,4.0,...,12494,94,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299218806
2,23949,997,4,1,14,13,8.0,8,8,4.0,...,12514,95,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299697786
3,23949,997,4,1,14,13,8.0,8,8,4.0,...,12534,96,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300176766
4,23949,997,4,1,14,13,8.0,8,8,4.0,...,12554,97,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300655746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352923,26085,1110,857,1,81,5,NaN,R,20,0.0,...,72085,1106,5.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Collision damage,1880337225
352924,26085,1110,857,1,81,5,NaN,R,20,0.0,...,72125,1107,5.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Collision damage,1881380625
352925,26085,1110,857,1,81,5,NaN,R,20,0.0,...,72145,1108,17.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Collision damage,1881902325
352926,26085,1110,857,1,81,5,NaN,R,20,0.0,...,72165,1109,27.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Collision damage,1882424025


In [340]:
p=df2['result_driver_standing']

In [341]:
df2.shape

(352928, 53)

In [342]:
p.shape

(352928,)

In [343]:
df2.isnull().sum()

resultId                       0
racerId                        0
driverId                       0
constructorId                  0
number                         0
grid                           0
position_x                 45237
positionText_x                 0
positionOrder                  0
points                         0
laps                           0
time_x                    145425
timetaken_in_millisec     145425
fastestLap                 12225
rank                           0
fastestLapTime             12225
max_speed                  12225
statusId                       0
year                           0
round                          0
circuitId                      0
grand_prix                     0
date                           0
time_y                         0
url_x                          0
fp1_date                  177237
fp1_time                  249976
fp2_date                  177237
fp2_time                  249976
fp3_date                  204987
fp3_time  

In [344]:
x = df2['timetaken_in_millisec'].mode()[0]
df2['timetaken_in_millisec'] = df2['timetaken_in_millisec'].fillna(x)
df2['timetaken_in_millisec'].isnull().sum()

0

In [345]:
x = df2['fastestLap'].mode()[0]
df2['fastestLap'] = df2['fastestLap'].fillna(x)
df2['fastestLap'].isnull().sum()

0

In [346]:
x = df2['fastestLapTime'].mode()[0]
df2['fastestLapTime'] = df2['fastestLapTime'].fillna(x)
df2['fastestLapTime'].isnull().sum()

0

In [347]:
x = df2['max_speed'].mean()
df2['max_speed'] = df2['max_speed'].fillna(x)
df2['max_speed'].isnull().sum()

0

In [348]:
df2=df2.drop(columns=['url_x','url_y','url','driver_num','driver_code','position_x','positionText_x','result_driver_standing','time_y','fp1_date','fp1_time','fp2_date','fp2_time','fp3_date','fp3_time','quali_date','quali_time','sprint_date','sprint_time','number','time_x'],axis=1)

In [349]:
df2.shape

(352928, 32)

In [350]:
df2['full_name'] = df2['forename'] + ' ' + df2['surname']

# Drop 'forename' and 'surname' columns if needed
df2 = df2.drop(columns=['forename', 'surname'])

In [351]:
from datetime import datetime
# Convert 'dob' to datetime format
df2['dob'] = pd.to_datetime(df2['dob'])

# Create a reference date for each race
df2['race_date'] = pd.to_datetime(df2['year'], format='%Y')

# Calculate age at the time of race
df2['age_at_race'] = df2['race_date'].dt.year - df2['dob'].dt.year

C:\Users\risha\AppData\Local\Temp\ipykernel_13404\3971766724.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['dob'] = pd.to_datetime(df2['dob'])


In [352]:
import category_encoders as ce

# Assuming df1 is your training dataset and df2 is your test dataset
# List of high-cardinality columns to encode
high_cardinality_columns = ['full_name', 'constructorRef', 'nationality_y','company','grand_prix','nationality','status']

# Initialize the target encoder
target_encoder = ce.TargetEncoder(cols=high_cardinality_columns)

# Fit target encoder on training data (df1)
target_encoder.fit(df1[high_cardinality_columns], df1['position'])

# Transform test data (df2) using the fitted encoder
df2_encoded = target_encoder.transform(df2[high_cardinality_columns])

# Replace original columns with encoded columns in df2
df2[high_cardinality_columns] = df2_encoded

# Now df2 contains the encoded values for high-cardinality columns based on training data


c:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\category_encoders\ordinal.py:198: Futur

In [353]:
# Columns to drop
columns_to_drop = ['resultId', 'racerId', 'driverId', 'constructorId','date', 'driverRef', 'driverStandingsId','positionText_y','year','raceId_y','statusId','position']

# Drop columns from df1 if they exist
df2 = df2.drop(columns=[col for col in columns_to_drop if col in df2.columns])

In [354]:
df2

,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLap,rank,fastestLapTime,max_speed,round,...,nationality,points_y,wins,constructorRef,company,nationality_y,status,full_name,race_date,age_at_race
0,13,8,4.0,70,5280899.0,69.0,9,01:08.7,226.399000,9,...,12.332001,16.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2018-01-01,37
1,13,8,4.0,70,5280899.0,69.0,9,01:08.7,226.399000,9,...,12.332001,21.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2018-01-01,37
2,13,8,4.0,70,5280899.0,69.0,9,01:08.7,226.399000,9,...,12.332001,21.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2018-01-01,37
3,13,8,4.0,70,5280899.0,69.0,9,01:08.7,226.399000,9,...,12.332001,25.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2018-01-01,37
4,13,8,4.0,70,5280899.0,69.0,9,01:08.7,226.399000,9,...,12.332001,25.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2018-01-01,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352923,5,20,0.0,0,5280899.0,53.0,0,01:32.8,210.857955,12,...,12.332001,5.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2023-01-01,22
352924,5,20,0.0,0,5280899.0,53.0,0,01:32.8,210.857955,12,...,12.332001,5.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2023-01-01,22
352925,5,20,0.0,0,5280899.0,53.0,0,01:32.8,210.857955,12,...,12.332001,17.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2023-01-01,22
352926,5,20,0.0,0,5280899.0,53.0,0,01:32.8,210.857955,12,...,12.332001,27.0,0,12.332001,12.332001,12.332001,12.332001,12.332001,2023-01-01,22


In [355]:
df2.isnull().sum()

grid                     0
positionOrder            0
points                   0
laps                     0
timetaken_in_millisec    0
fastestLap               0
rank                     0
fastestLapTime           0
max_speed                0
round                    0
circuitId                0
grand_prix               0
dob                      0
nationality              0
points_y                 0
wins                     0
constructorRef           0
company                  0
nationality_y            0
status                   0
full_name                0
race_date                0
age_at_race              0
dtype: int64

In [356]:
def convert_to_milliseconds(row):
    time_str = row['fastestLapTime']
    minutes, seconds = map(float, time_str.split(':'))
    return float((minutes * 60 + seconds)*1000)

df2['fastestLapTimeinMilliseconds'] = df2.apply(convert_to_milliseconds,axis=1)
df2['fastestLapTimeinMilliseconds']

0         68700.0
1         68700.0
2         68700.0
3         68700.0
4         68700.0
           ...   
352923    92800.0
352924    92800.0
352925    92800.0
352926    92800.0
352927    92800.0
Name: fastestLapTimeinMilliseconds, Length: 352928, dtype: float64

In [357]:
df2 = df2.drop(columns=['fastestLapTime'],axis=1)

In [358]:
df2.shape

(352928, 23)

In [359]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2264080 entries, 2752718 to 2219110
Data columns (total 21 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   grid                          int64  
 1   positionOrder                 int64  
 2   points                        float64
 3   laps                          int64  
 4   timetaken_in_millisec         float64
 5   fastestLap                    float64
 6   rank                          float64
 7   max_speed                     float64
 8   round                         int64  
 9   circuitId                     int64  
 10  grand_prix                    float64
 11  nationality                   float64
 12  points_y                      float64
 13  wins                          int64  
 14  constructorRef                float64
 15  company                       float64
 16  nationality_y                 float64
 17  status                        float64
 18  full_name            

In [360]:
df2=df2.drop(columns=['dob','race_date'])

In [361]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352928 entries, 0 to 352927
Data columns (total 21 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   grid                          352928 non-null  int64  
 1   positionOrder                 352928 non-null  int64  
 2   points                        352928 non-null  float64
 3   laps                          352928 non-null  int64  
 4   timetaken_in_millisec         352928 non-null  float64
 5   fastestLap                    352928 non-null  float64
 6   rank                          352928 non-null  int64  
 7   max_speed                     352928 non-null  float64
 8   round                         352928 non-null  int64  
 9   circuitId                     352928 non-null  int64  
 10  grand_prix                    352928 non-null  float64
 11  nationality                   352928 non-null  float64
 12  points_y                      352928 non-nul

In [362]:
df2.shape

(352928, 21)

In [363]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [364]:
x = df2[['fastestLapTimeinMilliseconds']]
scaler.fit(x)
df2['fastestLapTimeinMilliseconds'] = scaler.transform(x)
df2['fastestLapTimeinMilliseconds']

0         0.168996
1         0.168996
2         0.168996
3         0.168996
4         0.168996
            ...   
352923    0.475222
352924    0.475222
352925    0.475222
352926    0.475222
352927    0.475222
Name: fastestLapTimeinMilliseconds, Length: 352928, dtype: float64

In [365]:
x = df2[['timetaken_in_millisec']]
scaler.fit(x)
df2['timetaken_in_millisec'] = scaler.transform(x)
df2['timetaken_in_millisec']

0         0.46958
1         0.46958
2         0.46958
3         0.46958
4         0.46958
           ...   
352923    0.46958
352924    0.46958
352925    0.46958
352926    0.46958
352927    0.46958
Name: timetaken_in_millisec, Length: 352928, dtype: float64

In [366]:
y_pred = model.predict(df2)

In [367]:
y_pred

array([ 9.832319,  9.400382,  9.400382, ..., 13.102928, 12.291341,
        9.914736], dtype=float32)

In [368]:
y_pred.shape

(352928,)

In [369]:
p.shape

(352928,)

In [370]:
# Merge p (result_driver_standing) with y_pred
merged_df = pd.concat([p, pd.Series(y_pred, index=p.index)], axis=1)
merged_df.columns = ['result_driver_standing', 'position']

In [371]:
# Swap columns 'result_driver_standing' and 'p_position'
merged_df = merged_df[['position', 'result_driver_standing']]

In [372]:
merged_df

,position,result_driver_standing
0,9.832319,298739826
1,9.400382,299218806
2,9.400382,299697786
3,8.876740,300176766
4,8.876740,300655746
...,...,...
352923,13.005716,1880337225
352924,13.005716,1881380625
352925,13.102928,1881902325
352926,12.291341,1882424025


In [373]:
# Remove the index column before saving to CSV
merged_df.to_csv('submission.csv', index=False)